## 1️⃣ Check GPU and Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
!pip install -q torch torchvision captcha opencv-python tqdm

## 2️⃣ Setup Project Structure

In [ ]:
import os

# Create directories
os.makedirs('src', exist_ok=True)
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('models', exist_ok=True)

print("✓ Project structure created")

## 3️⃣ Upload Code Files

**Option A**: If you uploaded code as a Kaggle dataset, uncomment and run:
```python
!unzip -q /kaggle/input/captcha-breaker-code/captcha-code.zip -d .
```

**Option B**: Copy files from the cells below

In [ ]:
# If you have code dataset, uncomment:
# !unzip -q /kaggle/input/captcha-breaker-code/captcha-code.zip -d .
# !ls -la

### Create `src/__init__.py`

In [ ]:
%%writefile src/__init__.py
# Empty init file

### Create `generate_dataset.py`

⚠️ **Paste the full content from your local `generate_dataset.py` in the cell below**

In [ ]:
%%writefile generate_dataset.py
# TODO: Paste content from your generate_dataset.py here

### Create `preprocess.py`

⚠️ **Paste the full content from your local `preprocess.py` in the cell below**

In [ ]:
%%writefile preprocess.py
# TODO: Paste content from your preprocess.py here

### Create `src/model.py`

⚠️ **Paste the full content from your local `src/model.py` in the cell below**

In [ ]:
%%writefile src/model.py
# TODO: Paste content from your src/model.py here

### Create `train.py`

⚠️ **Paste the full content from your local `train.py` in the cell below**

In [ ]:
%%writefile train.py
# TODO: Paste content from your train.py here

### Create `predict.py`

⚠️ **Paste the full content from your local `predict.py` in the cell below**

In [ ]:
%%writefile predict.py
# TODO: Paste content from your predict.py here

## 4️⃣ Generate Dataset

Generate 10,000 synthetic CAPTCHA images

In [ ]:
!python generate_dataset.py

In [ ]:
# Check generated images
!ls -lh data/raw/ | head -10

## 5️⃣ Preprocess Images

Convert to grayscale and remove noise

In [ ]:
!python preprocess.py

In [ ]:
# Check preprocessed images
!ls -lh data/processed/ | head -10

## 6️⃣ Train the Model

This will take **15-30 minutes** on GPU.

Expected:
- Training accuracy should increase to 50-90%
- Loss should decrease steadily
- Much better than the 6.25% from broken bbox approach!

In [ ]:
!python train.py

## 7️⃣ Check Results

In [ ]:
import torch
import os

# Check model file
model_path = 'models/captcha_model.pth'
if os.path.exists(model_path):
    checkpoint = torch.load(model_path)
    file_size = os.path.getsize(model_path) / (1024 * 1024)
    print(f"✓ Model saved successfully: {model_path}")
    print(f"✓ File size: {file_size:.2f} MB")
else:
    print("✗ Model file not found!")

## 8️⃣ Test Predictions

In [ ]:
# Test on a few samples
import glob

sample_images = glob.glob('data/processed/*.png')[:5]

for img_path in sample_images:
    print(f"\nTesting: {img_path}")
    !python predict.py {img_path}

## 9️⃣ Visualize Sample Predictions

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
import string
from src.model import CTCCaptchaModel

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
characters = string.digits + string.ascii_uppercase

# Load model
model = CTCCaptchaModel(num_classes=len(characters))
model.load_state_dict(torch.load('models/captcha_model.pth', map_location=device))
model.to(device)
model.eval()

# Transform
transform = transforms.Compose([
    transforms.Resize((60, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Visualize 6 samples
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

sample_images = glob.glob('data/processed/*.png')[:6]

for idx, img_path in enumerate(sample_images):
    # Load image
    img = Image.open(img_path).convert('L')
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        pred_indices = model.predict(img_tensor)[0]
        pred_text = ''.join([characters[i] for i in pred_indices if i < len(characters)])
    
    # Get ground truth
    gt_text = os.path.basename(img_path).split('_')[0]
    
    # Plot
    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(f'GT: {gt_text}\nPred: {pred_text}', 
                       fontsize=12, 
                       color='green' if pred_text == gt_text else 'red')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Visualization saved to predictions.png")

## 🔟 Download Trained Model

In [ ]:
from IPython.display import FileLink

# Create download link
FileLink('models/captcha_model.pth')

## 1️⃣1️⃣ (Optional) Push to GitHub

**Setup GitHub Personal Access Token first:**
1. Go to GitHub → Settings → Developer settings → Personal access tokens
2. Generate new token with `repo` scope
3. Add to Kaggle: Notebook → Add-ons → Secrets → Add secret
   - Label: `GITHUB_TOKEN`
   - Value: your_github_token

In [ ]:
# Configure Git
!git config --global user.email "your.email@example.com"
!git config --global user.name "Your Name"

In [ ]:
# Setup GitHub authentication with token
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
github_token = user_secrets.get_secret("GITHUB_TOKEN")

# Clone repo (or use existing)
# Replace YOUR_USERNAME and YOUR_REPO
!git clone https://{github_token}@github.com/YOUR_USERNAME/captcha-breaker.git
%cd captcha-breaker

In [ ]:
# Copy trained model to repo
!cp ../models/captcha_model.pth models/

In [ ]:
# Commit and push
!git add models/captcha_model.pth
!git commit -m "Add trained CAPTCHA model from Kaggle"
!git push origin main

## ✅ Done!

Your model is now trained and ready to use.

**Next steps:**
1. Download the model using the FileLink above
2. Pull to your local machine: `git pull origin main`
3. Test locally: `python predict.py data/processed/test.png`